In [10]:
import re
import pandas as pd
import httpagentparser
from pprint import pprint
from datetime import datetime
import pytz
from pprint import pprint
from udger import Udger
import ipaddress
from geoip import geolite2
import urllib
import json,glob,os
udger = Udger()

In [11]:
def parse_str(x):
    """
    Returns the string delimited by two characters.

    Example:
        `>>> parse_str('[my string]')`
        `'my string'`
    """
    return x[1:-1]

def parse_datetime(x):
    '''
    Parses datetime with timezone formatted as:
        `[day/month/year:hour:minute:second zone]`

    Example:
        `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
        `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

    Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the 
    timezone will be obtained using the `pytz` library.
    '''    
    dt = datetime.strptime(x[1:-7], '%d/%b/%Y:%H:%M:%S')
    dt_tz = int(x[-6:-3])*60+int(x[-3:-1])    
    return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))

def get_ip_info(ip):
    url = 'http://ip-api.com/json/{0}'.format(ip)
    ip_info = json.loads(urllib.urlopen(url).read())
    if ip_info != None:
        return ip_info
    else:
        return {}

In [12]:
httpd_access_log_dir = "/home/ranjan/git/python-practice/logs/log/000000/"
logpath = "access_log-20140810"

In [14]:
log_files = glob.glob(os.path.join(httpd_access_log_dir,"access*"))
log_files

['/home/ranjan/git/python-practice/logs/log/000000/access-000000']

In [19]:
logs = pd.DataFrame()
list_df = []
for log_file in log_files:
    print(log_file)
    df = pd.read_csv(
    log_file, 
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])', 
    engine='python', 
    na_values='-', 
    header=None,
    usecols=[0, 3, 4, 5, 6, 7, 8],
    names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
    converters={'time': parse_datetime,
                'request': parse_str,
                'status': int,
                'size': int,
                'referer': parse_str,
                'user_agent': parse_str})    
    if df.empty:
        print('DataFrame is empty!')
    else:
        list_df.append(df)
logs = pd.concat(list_df)

/home/ranjan/git/python-practice/logs/log/000000/access-000000


ValueError: invalid literal for int() with base 10: '"GET /rest/user/position/user/84 HTTP/1.1"'

In [18]:
logs.head()

""


In [22]:
%matplotlib inline

In [10]:
request = logs.request.str.split()
logs['resource'] = request.str[1]
logs = logs[~logs.ip.isin(["127.0.0.1"])]

In [11]:
logs = logs[~logs['resource'].str.match(r'^/media|^/static|^/admin|^/robots.txt$|^/favicon.ico|^/myadmin|^/manager/ispmgr|^/webmail/$')]
logs = logs[~logs['user_agent'].str.match(r'.*?bot|.*?spider|.*?crawler|.*?slurp', flags=re.I).fillna(False)]

In [12]:
logs["browser"]=logs.user_agent.map(lambda x : httpagentparser.detect(x))
logs["ip_info"]=logs.ip.map(lambda x : get_ip_info(x))

In [15]:
logs.head(1)

,ip,time,request,status,size,referer,user_agent,resource,browser,ip_info,platform_name,platform_version,browser_name,browser_version,country,regionName
0,93.174.93.218,2014-08-09 03:18:05+00:00,GET http://httpheader.net HTTP/1.1,200,1575,NaN,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US...,http://httpheader.net,"{u'platform': {u'version': u'7', u'name': u'Wi...","{u'status': u'success', u'city': u'Amsterdam',...",Windows,7,Firefox,3.6.28,Netherlands,North Holland


In [14]:
logs["platform_name"]=logs.browser.map(lambda x : x.get("platform").get("name"))
logs["platform_version"]=logs.browser.map(lambda x : x.get("platform").get("version"))
logs["browser_name"]=logs.browser.map(lambda x : x.get("browser").get("name"))
logs["browser_version"]=logs.browser.map(lambda x : x.get("browser").get("version"))
logs["country"]=logs.ip_info.map(lambda x : x.get("country"))
logs["regionName"]=logs.ip_info.map(lambda x : x.get("regionName"))

In [24]:
logs.to_excel("/home/ranjan/git/python-practice/logs/access_log_sheet.xlsx")

In [9]:
httpagentparser.detect("ozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36")

{'bot': False,
 'browser': {'name': 'Chrome', 'version': '57.0.2987.133'},
 'os': {'name': 'Windows', 'version': '8'},
 'platform': {'name': 'Windows', 'version': '8'}}